In [1]:
from itertools import zip_longest
import polars as pl
from tqdm import tqdm

In [2]:
def load2df(file):
    with open(file) as fin:
        lines = fin.readlines()
        
    records = []
    tmp_record = {}
    is_first = True
    for line in tqdm(lines):
        line = line.replace("\n", "")

        first = ("|" in line)
        second = line.startswith("WEDNESDAY")
        third = line.startswith("THURSDAY")    

        # print(first, second, third)
        if first and (second or third):
            DAY = line.split("|")[0]
            TIME = line.split("|")[1]
            continue

        added_flag = False


        for key in ["Speaker:","Track:", "Tracks:", "Format:", "Location:", "Contributor:", "Panelist:", "Keynote:", "Moderator:", "Experience:"]:
            if line.startswith(key):
                assert key not in tmp_record, f"{line}, {key} {tmp_record}"
                added_key = key[:-1]

                if "Contributor:" == key:
                    added_key = "Speaker"
                if "Tracks:" == key:
                    added_key = "Track"
                if "Moderator" == key:
                    added_key = "Speaker"
                if "Keynote:" == key:
                    added_key = "Main"
                


                tmp_record[added_key] = ":".join(line.split(":")[1:])
                added_flag = True

        if not added_flag:
            assert "Main" not in tmp_record, f"{line} main {tmp_record}"
            tmp_record["Main"] = line

        if line.startswith("Location:"):
            tmp_record["Date"]= DAY
            tmp_record["Time"]= TIME
            records.append(tmp_record)

            tmp_record = {}
    return pl.DataFrame(records)

def load_session(file):
    with open(file, "r") as fin:
        return [a.strip() for a in fin.readlines()]    

In [3]:
df_2023 = load2df('2023_sessions.txt')
tracks_2023 = load_session('2023_tracks.txt')

100%|██████████| 590/590 [00:00<00:00, 239953.39it/s]


In [4]:
dict_2023={}
for target_label in tracks_2023:
    dict_2023[target_label] = df_2023.filter(pl.col("Track").str.contains(target_label)).shape[0]

sorted_keys_2023 = sorted(dict_2023, key=dict_2023.get, reverse=True)

In [19]:
sum_2023 = 0
for a in sorted_keys_2023:
    print(a, dict_2023[a])
    sum_2023 += dict_2023[a]

Defense 17
AI, ML, & Data Science 15
Exploit Development 14
Application Security: Offense 13
Enterprise Security 13
Hardware / Embedded 12
Application Security: Defense 11
Cloud Security 11
Data Forensics & Incident Response 11
Network Security 10
Platform Security 10
Cyber-Physical Systems & IoT 9
Mobile 8
Reverse Engineering 8
Cryptography 7
Malware 7
Policy 6
Human Factors 5
Cyber Insurance 4
Entrepreneur 4
Privacy 4
Community & Career 3
Keynote 3


AI, ML, & Data Science: 5 -> 15
    - 
    - 

Application Security: 17 → 24  
Enterprise Sesecurity: 17 → 24  
Data Forensics & Incident Response: 4 → 11   

Cyber insurance: 新規軸
    -　



In [6]:
df_2022 = load2df('2022_sessions.txt')
tracks_2022 = load_session('2022_tracks.txt')

100%|██████████| 659/659 [00:00<00:00, 249237.72it/s]


In [7]:
dict_2022={}
for target_label in tracks_2022:
    dict_2022[target_label] = df_2022.filter(pl.col("Track").str.contains(target_label)).shape[0]

sorted_keys_2022 = sorted(dict_2022, key=dict_2022.get, reverse=True)

In [20]:
sum_2022 = 0
for a in sorted_keys_2022:
    print(a, dict_2022[a])
    sum_2022 += dict_2022[a]

Cloud & Platform Security 21
Application Security 17
Defense 15
Exploit Development 15
Hardware / Embedded 14
Reverse Engineering 13
Network Security 12
Lessons Learned 9
Malware 9
Mobile 9
Enterprise Security 7
Cyber-Physical Systems 6
Human Factors 6
Policy 6
AI, ML, & Data Science 5
Cryptography 5
Privacy 5
Community & Career 4
Data Forensics & Incident Response 4
Keynote 3


In [9]:
df_2021 = load2df('2021_sessions.txt')
tracks_2021 = load_session('2021_tracks.txt')

100%|██████████| 669/669 [00:00<00:00, 239807.66it/s]


In [10]:
dict_2021={}
for target_label in tracks_2021:
    dict_2021[target_label] = df_2021.filter(pl.col("Track").str.contains(target_label)).shape[0]

sorted_keys_2021 = sorted(dict_2021, key=dict_2021.get, reverse=True)

In [30]:
sum_2021 = 0
for a in sorted_keys_2021:
    print(a, dict_2021[a])
    sum_2021 += dict_2021[a]

Cloud & Platform Security 25
Exploit Development 23
Reverse Engineering 18
Hardware / Embedded 15
Human Factors 14
Network Security 13
AppSec 12
Applied Security 12
Data Forensics & Incident Response 12
Defense 12
Malware 12
Community 10
CorpSec 10
Cyber-Physical Systems 10
Policy 10
Mobile 9
AI, ML, & Data Science 8
Cryptography 8
Keynote 4


In [29]:
df_2022.head()

Main,Speaker,Track,Location,Date,Time,Format,Experience
str,str,str,str,str,str,str,str
"""Breakfast""","""""","""""",""" Shoreline (So…","""WEDNESDAY """,""" 8:00AM""",null,null
""" Black Hat at …",""" Chris Krebs""",""" Keynote""",""" Oceanside (Le…","""WEDNESDAY """,""" 9:00AM""",""" 60-Minute Key…",null
"""AAD Joined Mac…",""" Mor Rubin""",""" Cloud & Plat…",""" Lagoon HI (Le…","""WEDNESDAY """,""" 10:20AM""",""" 40-Minute Bri…",null
"""All Your GNN M…",""" Yang Zhang, A…",""" AI, ML, & Da…",""" Jasmine (Leve…","""WEDNESDAY """,""" 10:20AM""",""" 40-Minute Bri…",""" In-Person/Vir…"
"""Automatic Prot…",""" Gabi Nakibly,…",""" Network Secu…",""" Lagoon KL (Le…","""WEDNESDAY """,""" 10:20AM""",""" 40-Minute Bri…",""" In-Person/Vir…"


In [27]:
print(df_2021.filter(pl.col("Track") != "").shape)
print(df_2022.filter(pl.col("Track") != "").shape)
print(df_2023.filter(pl.col("Track") != "").shape)

(131, 7)
(101, 8)
(109, 9)


In [28]:
df_2021.shape

(131, 7)